In [0]:
dbutils.widgets.text("Execution_date", "", "Execution Date (YYYY-MM-DD)")
execution_date = dbutils.widgets.get("Execution_date")
execution_date=int(execution_date.replace('-','').replace("'",""))
print(f"Running ETL for execution date: {execution_date}")

In [0]:

order_file_path=f'/Volumes/e_comm_data_modeling/default/bronze_e_comm_data/order_{execution_date}.csv'

# selling_file_path='/Volumes/e_comm_data_modeling/default/bronze_e_comm_data/bronze_sellers.csv'
#Celler data already inserted so no need to read again and again 

In [0]:
raw_order_df=spark.read.option('header',True)\
        .option('inferschema',True).csv(order_file_path) 
# raw_seller_df=spark.read.option('header',True)\
#     .option('inferschema',True).csv(selling_file_path) 
raw_order_df.count()

In [0]:
# raw_seller_df.printSchema()
raw_order_df.printSchema()

In [0]:
# Simple validations for bronze layer
from pyspark.sql import functions as F
raw_order_df=raw_order_df.dropna(how="all")\
    .filter(F.col('order_id').isNotNull() & F.col('customer_id').isNotNull() & F.col('seller_id').isNotNull() & F.col('address_id').isNotNull())

raw_order_df=raw_order_df.withColumn('order_date',
                                     F.when(F.col('order_date').rlike(r'^\d{4}-\d{2}-\d{2}'),
                                            F.to_date(F.col('order_date'),'yyyy-MM-dd'))\
                                                .when(F.col('order_date').rlike(r'^\d{2}-\d{2}-\d{4}'),
                                                      F.to_date(F.col('order_date'),'dd-MM-yyyy'))
                                                          .otherwise(None))

raw_order_df=raw_order_df.withColumn('delivery_date',
                                     F.when(F.col('delivery_date').rlike(r'^\d{4}-\d{2}-\d{2}'),
                                            F.to_date(F.col('delivery_date'),'yyyy-MM-dd'))\
                                                .when(F.col('delivery_date').rlike(r'^\d{2}-\d{2}-\d{4}'),
                                                      F.to_date(F.col('delivery_date'),'dd-MM-yyyy'))\
                                                          .otherwise(None))

# raw_seller_df=raw_seller_df.dropna(how="all")\
#     .filter(F.col('seller_id').isNotNull())

In [0]:
# raw_seller_df.printSchema()
# raw_order_df.printSchema()

In [0]:
%sql
-- Will craete a delta table which will hold daily order 
-- here we are going to append the data daily by partitioning the data by order_date

CREATE TABLE IF NOT EXISTS e_comm_data_modeling.default.raw_order (
  order_id STRING,
  product_id STRING,
  product_name STRING,
  category STRING,
  seller_id STRING,
  customer_id STRING,
  customer_name STRING,
  address_id STRING,
  order_date DATE,
  delivery_date DATE,
  quantity INT,
  unit_price INT,
  discount INT,
  is_cancelled INT,
  is_returned INT
) 
USING DELTA
TBLPROPERTIES(
  quality = 'bronze',
  partitioned_by = 'order_date'
)

In [0]:
raw_order_df.write.format('delta').mode('append').partitionBy('order_date').saveAsTable('e_comm_data_modeling.default.raw_order')
# raw_seller_df.write.format('delta').mode('overwrite').saveAsTable('e_comm_data_modeling.default.seller_data')

#.save()  -- it will only store the data will not keep any metadata about the file
#.saveasTable --it  will create the table and store into it and will maintain the metadata in metastore
#.insertInto() -- if u created the table already and u want to insert the data into the table go for it.

In [0]:
# raw_order_df.select('order_date').distinct().show()

In [0]:
# from delta.tables import *

# deltaTable = DeltaTable.forName(spark, 'e_comm_data_modeling.default.raw_order')

# fullHistoryDF = deltaTable.history()    # get the full history of the table
# fullHistoryDF.show()
# lastOperationDF = deltaTable.history(1) 
# lastOperationDF.show()
# df=spark.read.format('delta').option('versionAsOf', 2).table('e_comm_data_modeling.default.raw_order')

In [0]:
%sql
select order_date,count(1) from e_comm_data_modeling.default.raw_order
group by order_date;